In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df = pd.read_csv('Data/house_prices.csv')
df.sample(2)

,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
146527,146527,2 BHK Ready to Occupy Flat for sale Kasba Peth,"Kasba Peth, Pune has an appealing 2 BHK flat f...",66 Lac,7952.0,pune,NaN,Ready to Move,1 out of 3,Resale,...,NaN,NaN,NaN,1,NaN,NaN,NaN,830 sqft,NaN,NaN
155293,155293,2 BHK Ready to Occupy Flat for sale Sector 48,Creatively planned and constructed is a 2 BHK ...,31.5 Lac,3750.0,faridabad,800 sqft,Ready to Move,1 out of 4,New Property,...,North,"Garden/Park, Pool, Main Road",NaN,2,2,"1 Covered,",Freehold,NaN,NaN,NaN


In [33]:
df = df.iloc[:,1:] # Removed the Index column as it was unnecessary

In [34]:
df_clean = df.drop(['Society','Status','Car Parking','Super Area'],axis=1) # High Percentage of Missing Values: Society (58.5%), Car Parking (55.1%), and Super Area (57.4%). Also dropped the Status column as it has a constant value throughout

In [29]:
column_names = df.columns
print(column_names)

Index(['Title', 'Description', 'Amount(in rupees)', 'Price (in rupees)',
       'location', 'Carpet Area', 'Status', 'Floor', 'Transaction',
       'Furnishing', 'facing', 'overlooking', 'Society', 'Bathroom', 'Balcony',
       'Car Parking', 'Ownership', 'Super Area', 'Dimensions', 'Plot Area'],
      dtype='object')


In [35]:
df_cleaned = df[df['Amount(in rupees)'] != 'Call for Price']

In [36]:
def indian_str_to_number(s):
    s = s.strip()
    if s.endswith(' Lac'):
        num = float(s.replace('Lac', '').strip())
        return int(num * 1e5)
    elif s.endswith(' Cr'):
        num = float(s.replace('Cr', '').strip())
        return int(num * 1e7)
    else:
        return int(float(s))

df_cleaned.loc[:, 'Amount(in rupees)'] = df_cleaned['Amount(in rupees)'].apply(indian_str_to_number)

In [39]:
df_cleaned['Amount(in rupees)'].describe()

count      177847
unique       1559
top       8500000
freq         5264
Name: Amount(in rupees), dtype: int64

In [40]:
# Skewness value
print("Skewness:", df_cleaned['Amount(in rupees)'].skew())
# This shows extremely high skewness need to apply some transformations

Skewness: 270.25594003861767


In [53]:
# Applying log transformation
df_cleaned.loc[:, 'Log_Price'] = np.log1p(df_cleaned['Amount(in rupees)'].astype(float))

print("Skewness after log:", df_cleaned['Log_Price'].skew())

Skewness after log: 0.2887446822592996


In [44]:
df_cleaned = df_cleaned.drop('Price (in rupees)', axis=1)

In [45]:
df_cleaned.head(2)

,Title,Description,Amount(in rupees),location,Carpet Area,Status,Floor,Transaction,Furnishing,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area,Log_Price
0,1 BHK Ready to Occupy Flat for sale in Srushti...,"Bhiwandi, Thane has an attractive 1 BHK Flat f...",4200000,thane,500 sqft,Ready to Move,10 out of 11,Resale,Unfurnished,NaN,NaN,Srushti Siddhi Mangal Murti Complex,1,2,NaN,NaN,NaN,NaN,NaN,15.250595
1,2 BHK Ready to Occupy Flat for sale in Dosti V...,One can find this stunning 2 BHK flat for sale...,9800000,thane,473 sqft,Ready to Move,3 out of 22,Resale,Semi-Furnished,East,Garden/Park,Dosti Vihar,2,NaN,1 Open,Freehold,NaN,NaN,NaN,16.097893


In [46]:
# Extracting number of rooms from the title column
df_cleaned['BHK'] = df_cleaned['Title'].str.extract('(\d+)').astype(float)

In [52]:
# Handling missing values
df_cleaned['BHK'].isnull().sum() # 258
# Its better to use median instead of mean because the mean (average) is sensitive to outliers. Imagine your data has a few 10-BHK mansions. These large values will pull the mean higher, making it less representative of a "typical" house.

# 1. Calculate the median of the BHK column
bhk_median = df_cleaned['BHK'].median()
print(f"The median BHK is: {bhk_median}")

# 2. Fill the missing values (NaNs) with the median
df_cleaned['BHK'] = df_cleaned['BHK'].fillna(bhk_median)

# You can verify that there are no more nulls
print(f"Null values in BHK after filling: {df_cleaned['BHK'].isnull().sum()}")

The median BHK is: 3.0
Null values in BHK after filling: 0


In [54]:
df_cleaned = df_cleaned.drop('Title', axis=1)

In [56]:
cols_to_fill = ['facing', 'overlooking', 'Balcony']  # replace with your columns

for col in cols_to_fill:
    df_cleaned[col]=df_cleaned[col].fillna(df_cleaned[col].mode()[0])

In [57]:
df_cleaned['Floor'] = df_cleaned['Floor'].str.extract('(\d+)').astype(float)

In [58]:
df_cleaned['Floor'].describe()

count    170893.000000
mean          4.841954
std           4.645418
min           1.000000
25%           2.000000
50%           3.000000
75%           6.000000
max         200.000000
Name: Floor, dtype: float64

In [67]:
print(df_cleaned['Floor'].skew())

2.8631639562125817


In [60]:
df_cleaned['Carpet Area'].describe()

count        101522
unique         2699
top       1000 sqft
freq           5223
Name: Carpet Area, dtype: object

In [61]:
df_cleaned['Area'] = df_cleaned['Carpet Area'].str.extract(r'(\d+\.?\d*)').astype(float)

In [63]:
print(df_cleaned['Area'].skew())

165.82480607951786


In [64]:
df_cleaned['Area'] = np.log1p(df_cleaned['Area'])

In [65]:
print(df_cleaned['Area'].skew())

-1.5989631331705665


In [68]:
df_cleaned = df_cleaned.drop('Carpet Area', axis=1)

In [69]:
df_cleaned.head(0)

,Description,Amount(in rupees),location,Status,Floor,Transaction,Furnishing,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area,Log_Price,BHK,Area


In [70]:
print(f"Number of unique locations: {df_cleaned['location'].nunique()}")

Number of unique locations: 81


In [71]:
location_counts = df_cleaned['location'].value_counts()
print(location_counts)

location
new-delhi      24945
bangalore      23262
kolkata        21605
gurgaon        18846
ahmedabad      12614
               ...  
solapur           30
ahmadnagar        30
nellore           29
pondicherry       29
madurai           26
Name: count, Length: 81, dtype: int64


In [74]:
locations_to_group = location_counts[location_counts < 40].index

In [75]:
df_cleaned.loc[df_cleaned['location'].isin(locations_to_group), 'location'] = 'Other'

print(f"New number of unique locations after grouping: {df_cleaned['location'].nunique()}")

New number of unique locations after grouping: 75


In [76]:
df_encoded = pd.get_dummies(df_cleaned, columns=['location','Transaction','Furnishing','Ownership'], drop_first=True)

In [78]:
df_encoded.head(2)

,Description,Amount(in rupees),Status,Floor,facing,overlooking,Society,Bathroom,Balcony,Car Parking,...,location_vrindavan,location_zirakpur,Transaction_Other,Transaction_Rent/Lease,Transaction_Resale,Furnishing_Semi-Furnished,Furnishing_Unfurnished,Ownership_Freehold,Ownership_Leasehold,Ownership_Power Of Attorney
0,"Bhiwandi, Thane has an attractive 1 BHK Flat f...",4200000,Ready to Move,10.0,East,Main Road,Srushti Siddhi Mangal Murti Complex,1,2,NaN,...,False,False,False,False,True,False,True,False,False,False
1,One can find this stunning 2 BHK flat for sale...,9800000,Ready to Move,3.0,East,Garden/Park,Dosti Vihar,2,2,1 Open,...,False,False,False,False,True,True,False,True,False,False


In [79]:
df_encoded = df_encoded.drop(['Description','Status','Society'],axis=1)

In [80]:
df_encoded =  pd.get_dummies(df_encoded, columns=['facing'], drop_first=True)

In [84]:
df_encoded =  df_encoded.drop(['overlooking','Car Parking'], axis=1)

In [85]:
df_encoded = df_encoded.drop(['Amount(in rupees)','Super Area','Dimensions','Plot Area'],axis=1)

In [86]:
df_encoded.to_csv('Cleaned_data.csv')